In [1]:
import numpy as np
import pandas as pd
import csv
import re
import json
from sklearn.impute import KNNImputer

In [2]:
# A cancer methylation data processing

In [3]:
import os,shutil
wenjian=r"../data/tumor/"
Methylation_name=[]
root_Methylation_name=[]
for root,dirs,files in os.walk(wenjian):
    for f in files:
        if 'level3betas.txt' in f and '.parcel' not in f:
            Methylation_name.append(f)
            f=os.path.join(root,f)
            root_Methylation_name.append(f)

In [4]:
len(Methylation_name) # 352
Methylation_name
len(Methylation_name) # 352
root_Methylation_name

['../data/COAD/4f503c6d-43d5-4ad9-b916-18621e96d001/78f4fcdb-32eb-46a9-8930-4a6e979f73de.methylation_array.sesame.level3betas.txt',
 '../data/COAD/0316705a-529c-4de8-8b66-b0c8cb2136e8/d101419a-fc14-40f9-9d2e-75421de375d8.methylation_array.sesame.level3betas.txt',
 '../data/COAD/28bffe7d-e282-4b07-925e-284708d21252/c4dd7ee5-3dd9-4722-8217-78362ea233ef.methylation_array.sesame.level3betas.txt',
 '../data/COAD/a9a682e2-fdd2-48d6-b2fa-fc8cb5d0cf30/791c85d1-e1ee-41de-b3d4-f289796e8477.methylation_array.sesame.level3betas.txt',
 '../data/COAD/06f61b76-aee6-4efd-b94e-2720c5523bec/b812e9fa-f10f-4951-b021-851373ec7d16.methylation_array.sesame.level3betas.txt',
 '../data/COAD/c4f0b164-91f4-40e8-947c-e39553e3c2a1/d1887a72-d79f-406c-b91e-56ab24ef8733.methylation_array.sesame.level3betas.txt',
 '../data/COAD/d7278d15-ceed-4867-800a-69448afb349f/3d084dd4-b696-42a2-90f3-3ed650aac6e4.methylation_array.sesame.level3betas.txt',
 '../data/COAD/37d9d16c-f222-49c5-b5f4-e059cbae9bcd/fd4f2a15-eee0-43c4-9423-

In [5]:
# Get COAD's metadata.json file for matching sample names
with open('../data/COAD_metadata.json','r',encoding='utf8')as fp:
    json_Methylation = json.load(fp)
len(json_Methylation) # 352
# json_Methylation

352

In [6]:
# get the result that the sample name matches the file name
Methylation_name_list = []
tmp = []
for i in json_Methylation:
    tmp = i["associated_entities"]
    tmp1 = tmp[0]
    Methylation_name_list.append([tmp1["entity_submitter_id"],i["file_name"]])
len(Methylation_name_list) # 352
Methylation_name_list

[['TCGA-D5-6530-01A-11D-1721-05',
  '750099e6-a36c-4747-a774-e4e9826da7b5.methylation_array.sesame.level3betas.txt'],
 ['TCGA-AA-3660-11A-01D-1721-05',
  '55e32e08-3ea1-4a0c-9964-a3a563c2db4b.methylation_array.sesame.level3betas.txt'],
 ['TCGA-DM-A28G-01A-11D-A16X-05',
  'd694bc9b-403c-40bb-8964-231cce7db4a4.methylation_array.sesame.level3betas.txt'],
 ['TCGA-AA-3660-01A-01D-1721-05',
  'a1c1430d-ed16-480e-9902-baa9f5424eea.methylation_array.sesame.level3betas.txt'],
 ['TCGA-G4-6307-01A-11D-1721-05',
  '92f8d7a1-0b3a-4b2f-bfb7-79e1935abb5f.methylation_array.sesame.level3betas.txt'],
 ['TCGA-AA-3511-01A-21D-1837-05',
  '77a67517-14c5-4311-a8f8-db9c6d9f1656.methylation_array.sesame.level3betas.txt'],
 ['TCGA-D5-5538-01A-01D-1651-05',
  '9bd8cd34-2aa0-487b-8aa4-de8ed86dffeb.methylation_array.sesame.level3betas.txt'],
 ['TCGA-CA-5255-01A-11D-1837-05',
  'a939c3ab-4320-493b-8676-6c41be0a281c.methylation_array.sesame.level3betas.txt'],
 ['TCGA-NH-A5IV-01A-42D-A36Y-05',
  'd1887a72-d79f-406c-

In [7]:
# Find a txt file from root_coad_name to get the feature name and save it to a csv file
file = "../data/tumor/4f503c6d-43d5-4ad9-b916-18621e96d001/78f4fcdb-32eb-46a9-8930-4a6e979f73de.methylation_array.sesame.level3betas.txt"
feature = pd.read_csv(file, sep='\t',header=None)
feature.columns = ["name","Beta value"]
feature.to_csv("../data/COAD_feature_name.csv", sep=',', encoding='utf-8')

In [8]:
# Find data based on root_Methylation_name

In [9]:
Methylation_data = pd.read_csv("../data/COAD_feature_name.csv",usecols=['name'])
Methylation_data

,name
0,cg07549526
1,cg16670573
2,cg09969830
3,cg00179196
4,cg03948744
...,...
485572,cg03930849
485573,cg17939569
485574,cg13365400
485575,cg21106100


In [10]:
for file in root_Methylation_name:
    f = pd.read_csv(file, sep='\t',header=None,usecols=[1])
    Methylation_data = pd.concat([Methylation_data,f],axis = 1)

In [11]:
Methylation_data

,name,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
0,cg07549526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cg16670573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cg09969830,0.014043,0.012550,0.011653,0.013177,0.012054,0.014371,0.017740,0.011676,0.012085,...,0.016071,0.010827,0.010989,0.011014,0.017859,0.015800,0.015870,0.014479,0.010706,0.014464
3,cg00179196,0.972344,0.887519,0.978516,0.972183,0.973728,0.961363,0.961283,0.969349,0.967820,...,0.971062,0.934276,0.979112,0.969353,0.952880,0.895301,0.961657,0.970165,0.977964,0.966383
4,cg03948744,0.035103,0.034084,0.021693,0.028394,0.044442,0.041549,0.059346,0.026885,0.035369,...,0.040218,0.026272,0.033357,0.028330,0.037437,0.032649,0.046022,0.037043,0.026631,0.034503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485572,cg03930849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
485573,cg17939569,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
485574,cg13365400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
485575,cg21106100,0.947265,0.932040,0.955336,NaN,0.916020,NaN,0.942137,NaN,NaN,...,0.815106,0.940617,0.463380,NaN,0.910882,NaN,NaN,NaN,0.958745,0.894105


In [12]:
Methylation_data.set_index(["name"], inplace=True)
Methylation_data

,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
name,,,,,,,,,,,,,,,,,,,,,
cg07549526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg16670573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg09969830,0.014043,0.012550,0.011653,0.013177,0.012054,0.014371,0.017740,0.011676,0.012085,0.014015,...,0.016071,0.010827,0.010989,0.011014,0.017859,0.015800,0.015870,0.014479,0.010706,0.014464
cg00179196,0.972344,0.887519,0.978516,0.972183,0.973728,0.961363,0.961283,0.969349,0.967820,0.945965,...,0.971062,0.934276,0.979112,0.969353,0.952880,0.895301,0.961657,0.970165,0.977964,0.966383
cg03948744,0.035103,0.034084,0.021693,0.028394,0.044442,0.041549,0.059346,0.026885,0.035369,0.032455,...,0.040218,0.026272,0.033357,0.028330,0.037437,0.032649,0.046022,0.037043,0.026631,0.034503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cg03930849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg17939569,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg13365400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Because we will get the data according to the order of root_Methylation_name, and root_Methylation_name is in the same order as Methylation_name
# So we need to reorder the sample names of Methylation_name_list in the order of Methylation_name
feature_name = []
for i in Methylation_name:
    for j in range(len(Methylation_name_list)):
        if i == Methylation_name_list[j][1]:
            feature_name.append(Methylation_name_list[j][0])

In [14]:
feature_name
len(feature_name)

352

In [15]:
# Use regular expression to take the first few of the tumor
# Normally, just take the first four of the previous sample name, you can check the sample name definition for details.
pattern = re.compile(r"(TCGA-\w*-\w*-\w*)")  
feature_name_list = []
for strs in feature_name:
    test_strs = "".join(strs)
    result = pattern.findall(test_strs)
    feature_name_list.append("".join(result))

In [16]:
feature_name_list
len(feature_name_list) # 352

352

In [17]:
# Use the obtained feature_name_list as the column name of Methylation_data, that is, the sample name

In [18]:
Methylation_data.columns = feature_name_list

In [19]:
Methylation_data

,TCGA-A6-6141-01A,TCGA-AZ-6599-11A,TCGA-G4-6315-01A,TCGA-AZ-6608-01A,TCGA-AY-A8YK-01A,TCGA-NH-A5IV-01A,TCGA-D5-5537-01A,TCGA-DM-A0XF-01A,TCGA-A6-6650-01A,TCGA-A6-2671-11A,...,TCGA-A6-6137-01A,TCGA-AY-6196-01A,TCGA-DM-A28A-01A,TCGA-G4-6317-01A,TCGA-CM-6169-01A,TCGA-A6-4107-11A,TCGA-A6-2686-01A,TCGA-DM-A285-01A,TCGA-DM-A28C-01A,TCGA-QG-A5Z1-01A
name,,,,,,,,,,,,,,,,,,,,,
cg07549526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg16670573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg09969830,0.014043,0.012550,0.011653,0.013177,0.012054,0.014371,0.017740,0.011676,0.012085,0.014015,...,0.016071,0.010827,0.010989,0.011014,0.017859,0.015800,0.015870,0.014479,0.010706,0.014464
cg00179196,0.972344,0.887519,0.978516,0.972183,0.973728,0.961363,0.961283,0.969349,0.967820,0.945965,...,0.971062,0.934276,0.979112,0.969353,0.952880,0.895301,0.961657,0.970165,0.977964,0.966383
cg03948744,0.035103,0.034084,0.021693,0.028394,0.044442,0.041549,0.059346,0.026885,0.035369,0.032455,...,0.040218,0.026272,0.033357,0.028330,0.037437,0.032649,0.046022,0.037043,0.026631,0.034503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cg03930849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg17939569,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg13365400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# find samples with msi
data_MSI = pd.read_csv(r"../data/maf/COAD_data_MSI.csv",sep = ',',encoding = 'utf-8',usecols=['simple_tumor4','Predicted_MSI_Status'])
data_MSI

,Predicted_MSI_Status,simple_tumor4
0,MSI-H,TCGA-A6-6781-01A
1,MSS,TCGA-CM-6172-01A
2,MSS,TCGA-DM-A0XD-01A
3,MSS,TCGA-AA-3655-01A
4,MSS,TCGA-DM-A1HA-01A
...,...,...
449,MSI-H,TCGA-CK-6746-01A
450,MSS,TCGA-CM-6676-01A
451,MSS,TCGA-AA-A02E-01A
452,MSS,TCGA-DM-A1DA-01A


In [21]:
# Convert the above dataframe into a list format so that you can use regular expressions to take out the sample names
data_MSI_list = data_MSI.values.tolist()
# len(data_MSI_list) # 454
data_MSI_list

[['MSI-H', 'TCGA-A6-6781-01A'],
 ['MSS', 'TCGA-CM-6172-01A'],
 ['MSS', 'TCGA-DM-A0XD-01A'],
 ['MSS', 'TCGA-AA-3655-01A'],
 ['MSS', 'TCGA-DM-A1HA-01A'],
 ['MSS', 'TCGA-D5-7000-01A'],
 ['MSI-H', 'TCGA-F4-6856-01A'],
 ['MSS', 'TCGA-AA-3667-01A'],
 ['MSS', 'TCGA-D5-6538-01A'],
 ['MSS', 'TCGA-AZ-6600-01A'],
 ['MSS', 'TCGA-G4-6317-01A'],
 ['MSS', 'TCGA-AZ-4315-01A'],
 ['MSS', 'TCGA-AA-3955-01A'],
 ['MSS', 'TCGA-AA-3520-01A'],
 ['MSS', 'TCGA-CM-6168-01A'],
 ['MSI-H', 'TCGA-AA-3811-01A'],
 ['MSS', 'TCGA-AY-6196-01A'],
 ['MSS', 'TCGA-CM-6680-01A'],
 ['MSS', 'TCGA-DM-A0XF-01A'],
 ['MSS', 'TCGA-CM-6170-01A'],
 ['MSS', 'TCGA-AA-3939-01A'],
 ['MSS', 'TCGA-F4-6569-01A'],
 ['MSI-H', 'TCGA-A6-2672-01B'],
 ['MSS', 'TCGA-AY-A69D-01A'],
 ['MSS', 'TCGA-AA-3842-01A'],
 ['MSI-H', 'TCGA-A6-A565-01A'],
 ['MSI-H', 'TCGA-AA-3672-01A'],
 ['MSS', 'TCGA-A6-5660-01A'],
 ['MSS', 'TCGA-D5-6932-01A'],
 ['MSS', 'TCGA-AA-3814-01A'],
 ['MSS', 'TCGA-CM-6166-01A'],
 ['MSS', 'TCGA-F4-6461-01A'],
 ['MSI-H', 'TCGA-D5-6928-01A

In [22]:
# Use regular expression to take the first few of the tumor
# Normally, just take the first four of the previous sample name, you can check the sample name definition for details.
pattern = re.compile(r"(TCGA-\w*-\w*-\w*)$")  
MSI_list = []
for strs in data_MSI_list:
    test_strs = "".join(strs)
    result = pattern.findall(test_strs)
    MSI_list.append("".join(result))

In [23]:
len(MSI_list) # 454
MSI_list

['TCGA-A6-6781-01A',
 'TCGA-CM-6172-01A',
 'TCGA-DM-A0XD-01A',
 'TCGA-AA-3655-01A',
 'TCGA-DM-A1HA-01A',
 'TCGA-D5-7000-01A',
 'TCGA-F4-6856-01A',
 'TCGA-AA-3667-01A',
 'TCGA-D5-6538-01A',
 'TCGA-AZ-6600-01A',
 'TCGA-G4-6317-01A',
 'TCGA-AZ-4315-01A',
 'TCGA-AA-3955-01A',
 'TCGA-AA-3520-01A',
 'TCGA-CM-6168-01A',
 'TCGA-AA-3811-01A',
 'TCGA-AY-6196-01A',
 'TCGA-CM-6680-01A',
 'TCGA-DM-A0XF-01A',
 'TCGA-CM-6170-01A',
 'TCGA-AA-3939-01A',
 'TCGA-F4-6569-01A',
 'TCGA-A6-2672-01B',
 'TCGA-AY-A69D-01A',
 'TCGA-AA-3842-01A',
 'TCGA-A6-A565-01A',
 'TCGA-AA-3672-01A',
 'TCGA-A6-5660-01A',
 'TCGA-D5-6932-01A',
 'TCGA-AA-3814-01A',
 'TCGA-CM-6166-01A',
 'TCGA-F4-6461-01A',
 'TCGA-D5-6928-01A',
 'TCGA-A6-5665-01B',
 'TCGA-AD-A5EJ-01A',
 'TCGA-A6-6137-01A',
 'TCGA-G4-6299-01A',
 'TCGA-CM-5862-01A',
 'TCGA-AA-3514-01A',
 'TCGA-D5-6532-01A',
 'TCGA-DM-A0X9-01A',
 'TCGA-A6-6651-01A',
 'TCGA-AA-A02H-01A',
 'TCGA-D5-6929-01A',
 'TCGA-AA-3697-01A',
 'TCGA-A6-3809-01B',
 'TCGA-NH-A50U-01A',
 'TCGA-AD-A5E

In [24]:
# Find out the samples related to msi in feature_name_list

In [25]:
# feature_name_list 352 MSI_list 454
merge_name = [i for i in feature_name_list if i in MSI_list]
merge_name
len(merge_name)# 308

308

In [26]:
# Check how many duplicates feature_name_list has
dif_sample = []
same_sample = []
for i in merge_name:
    if i not in dif_sample:
        dif_sample.append(i)
    elif i in dif_sample:
        same_sample.append(i)

In [27]:
same_sample

['TCGA-A6-6650-01A',
 'TCGA-A6-5656-01A',
 'TCGA-A6-6781-01A',
 'TCGA-A6-2684-01A',
 'TCGA-A6-5659-01A',
 'TCGA-A6-6780-01A']

In [28]:
# Because you don't want samples with the same name in the future, so delete them directly
merge_name1 = []
for i in merge_name:
    if i not in same_sample:
        merge_name1.append(i)
merge_name = merge_name1

In [29]:
# Take the msi-tagged sample data from the total according to merge_name

In [30]:
new_data0 = Methylation_data[merge_name]
new_data0

,TCGA-G4-6315-01A,TCGA-AZ-6608-01A,TCGA-AY-A8YK-01A,TCGA-NH-A5IV-01A,TCGA-D5-5537-01A,TCGA-DM-A0XF-01A,TCGA-D5-6539-01A,TCGA-F4-6570-01A,TCGA-D5-6926-01A,TCGA-CM-6167-01A,...,TCGA-AD-6548-01A,TCGA-A6-6137-01A,TCGA-AY-6196-01A,TCGA-DM-A28A-01A,TCGA-G4-6317-01A,TCGA-CM-6169-01A,TCGA-A6-2686-01A,TCGA-DM-A285-01A,TCGA-DM-A28C-01A,TCGA-QG-A5Z1-01A
name,,,,,,,,,,,,,,,,,,,,,
cg07549526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg16670573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg09969830,0.011653,0.013177,0.012054,0.014371,0.017740,0.011676,0.010652,0.009236,0.016179,0.011976,...,0.014135,0.016071,0.010827,0.010989,0.011014,0.017859,0.015870,0.014479,0.010706,0.014464
cg00179196,0.978516,0.972183,0.973728,0.961363,0.961283,0.969349,0.957642,0.970408,0.954551,0.937093,...,0.962125,0.971062,0.934276,0.979112,0.969353,0.952880,0.961657,0.970165,0.977964,0.966383
cg03948744,0.021693,0.028394,0.044442,0.041549,0.059346,0.026885,0.027980,0.034306,0.041131,0.039667,...,0.032936,0.040218,0.026272,0.033357,0.028330,0.037437,0.046022,0.037043,0.026631,0.034503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cg03930849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg17939569,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg13365400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# Transpose new_data
new_data1 = pd.DataFrame(new_data0.values.T, index=new_data0.columns, columns=new_data0.index)#转置
new_data1

name,cg07549526,cg16670573,cg09969830,cg00179196,cg03948744,cg02729269,cg10009236,cg10143220,cg05791870,cg01527023,...,cg00335297,cg03827298,cg22051787,cg01757887,cg00061679,cg03930849,cg17939569,cg13365400,cg21106100,cg14273923
TCGA-G4-6315-01A,NaN,NaN,0.011653,0.978516,0.021693,0.147438,0.014975,0.939172,0.987482,0.015901,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.955336,0.910819
TCGA-AZ-6608-01A,NaN,NaN,0.013177,0.972183,0.028394,0.180064,0.019211,0.962210,0.980854,0.016335,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-AY-A8YK-01A,NaN,NaN,0.012054,0.973728,0.044442,0.107630,0.016470,0.962572,0.984566,0.016827,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.916020,0.901096
TCGA-NH-A5IV-01A,NaN,NaN,0.014371,0.961363,0.041549,0.160063,0.017490,0.901776,0.980618,0.026201,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-D5-5537-01A,NaN,NaN,0.017740,0.961283,0.059346,0.256496,0.029298,0.964023,0.976251,0.022938,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.942137,0.915780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-CM-6169-01A,NaN,NaN,0.017859,0.952880,0.037437,0.154822,0.024580,0.930985,0.979309,0.021301,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.910882,0.814834
TCGA-A6-2686-01A,NaN,NaN,0.015870,0.961657,0.046022,0.194068,0.021561,0.953753,0.982934,0.020668,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-DM-A285-01A,NaN,NaN,0.014479,0.970165,0.037043,0.185207,0.019798,0.907580,0.984501,0.019738,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-DM-A28C-01A,NaN,NaN,0.010706,0.977964,0.026631,0.202950,0.016957,0.952098,0.987829,0.021578,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.958745,0.885484


In [32]:
# Delete the null value feature, as long as a column has a null value, delete it directly
new_data1.dropna(axis=1, how='all', thresh=None, subset=None, inplace=True)
new_data1

name,cg09969830,cg00179196,cg03948744,cg02729269,cg10009236,cg10143220,cg05791870,cg01527023,cg00928894,cg02369618,...,cg14029254,cg02352633,cg05051262,cg10666546,cg25640065,cg25918849,cg11021362,cg25914522,cg21106100,cg14273923
TCGA-G4-6315-01A,0.011653,0.978516,0.021693,0.147438,0.014975,0.939172,0.987482,0.015901,0.067963,0.014555,...,0.908654,0.901336,0.892368,0.954866,0.955910,0.212444,0.929874,0.939430,0.955336,0.910819
TCGA-AZ-6608-01A,0.013177,0.972183,0.028394,0.180064,0.019211,0.962210,0.980854,0.016335,0.079980,0.014075,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-AY-A8YK-01A,0.012054,0.973728,0.044442,0.107630,0.016470,0.962572,0.984566,0.016827,0.091078,0.016981,...,0.904180,NaN,0.887747,0.943232,0.902062,0.132793,0.867440,0.948413,0.916020,0.901096
TCGA-NH-A5IV-01A,0.014371,0.961363,0.041549,0.160063,0.017490,0.901776,0.980618,0.026201,0.112854,0.019446,...,NaN,NaN,NaN,NaN,0.703756,NaN,NaN,NaN,NaN,NaN
TCGA-D5-5537-01A,0.017740,0.961283,0.059346,0.256496,0.029298,0.964023,0.976251,0.022938,0.061267,0.024531,...,0.679239,NaN,0.556286,0.961690,0.921391,0.107743,0.864028,0.917506,0.942137,0.915780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-CM-6169-01A,0.017859,0.952880,0.037437,0.154822,0.024580,0.930985,0.979309,0.021301,0.063823,0.017074,...,0.836950,NaN,0.810845,0.939489,0.841554,0.266704,NaN,0.919844,0.910882,0.814834
TCGA-A6-2686-01A,0.015870,0.961657,0.046022,0.194068,0.021561,0.953753,0.982934,0.020668,0.074079,0.019188,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-DM-A285-01A,0.014479,0.970165,0.037043,0.185207,0.019798,0.907580,0.984501,0.019738,0.116022,0.019065,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-DM-A28C-01A,0.010706,0.977964,0.026631,0.202950,0.016957,0.952098,0.987829,0.021578,0.084261,0.013433,...,0.960349,0.908695,0.884650,0.965297,0.951437,0.091617,0.914033,0.941300,0.958745,0.885484


In [33]:
# The nearest 10 neighbors, using the mean of the 10
tmp = new_data1
impute = KNNImputer(n_neighbors=10) 
tmp = impute.fit_transform(tmp)
tmp = pd.DataFrame(tmp)
sample_rowname = new_data1._stat_axis.values.tolist() 
sample_listname = new_data1.columns.values.tolist() 
tmp.index = sample_rowname
tmp.columns = sample_listname
new_data1 = tmp
new_data1

,cg09969830,cg00179196,cg03948744,cg02729269,cg10009236,cg10143220,cg05791870,cg01527023,cg00928894,cg02369618,...,cg14029254,cg02352633,cg05051262,cg10666546,cg25640065,cg25918849,cg11021362,cg25914522,cg21106100,cg14273923
TCGA-G4-6315-01A,0.011653,0.978516,0.021693,0.147438,0.014975,0.939172,0.987482,0.015901,0.067963,0.014555,...,0.908654,0.901336,0.892368,0.954866,0.955910,0.212444,0.929874,0.939430,0.955336,0.910819
TCGA-AZ-6608-01A,0.013177,0.972183,0.028394,0.180064,0.019211,0.962210,0.980854,0.016335,0.079980,0.014075,...,0.788584,0.903834,0.778873,0.962001,0.927002,0.199467,0.904817,0.817178,0.831016,0.868006
TCGA-AY-A8YK-01A,0.012054,0.973728,0.044442,0.107630,0.016470,0.962572,0.984566,0.016827,0.091078,0.016981,...,0.904180,0.878067,0.887747,0.943232,0.902062,0.132793,0.867440,0.948413,0.916020,0.901096
TCGA-NH-A5IV-01A,0.014371,0.961363,0.041549,0.160063,0.017490,0.901776,0.980618,0.026201,0.112854,0.019446,...,0.816789,0.896285,0.803585,0.963357,0.703756,0.177622,0.892458,0.798545,0.912893,0.871487
TCGA-D5-5537-01A,0.017740,0.961283,0.059346,0.256496,0.029298,0.964023,0.976251,0.022938,0.061267,0.024531,...,0.679239,0.893329,0.556286,0.961690,0.921391,0.107743,0.864028,0.917506,0.942137,0.915780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-CM-6169-01A,0.017859,0.952880,0.037437,0.154822,0.024580,0.930985,0.979309,0.021301,0.063823,0.017074,...,0.836950,0.885340,0.810845,0.939489,0.841554,0.266704,0.877621,0.919844,0.910882,0.814834
TCGA-A6-2686-01A,0.015870,0.961657,0.046022,0.194068,0.021561,0.953753,0.982934,0.020668,0.074079,0.019188,...,0.845077,0.882007,0.743532,0.923484,0.921326,0.157803,0.894761,0.744305,0.913533,0.862811
TCGA-DM-A285-01A,0.014479,0.970165,0.037043,0.185207,0.019798,0.907580,0.984501,0.019738,0.116022,0.019065,...,0.781015,0.881488,0.779299,0.958641,0.895560,0.204045,0.884583,0.665550,0.899826,0.859715
TCGA-DM-A28C-01A,0.010706,0.977964,0.026631,0.202950,0.016957,0.952098,0.987829,0.021578,0.084261,0.013433,...,0.960349,0.908695,0.884650,0.965297,0.951437,0.091617,0.914033,0.941300,0.958745,0.885484


In [34]:
# In addition to obtaining sample data, the corresponding msi label is also required, and the sample data needs to be matched with the corresponding label

In [37]:
# First obtain the tuple corresponding to the sample label and msi, and then sort according to the current label data
dif_MSI_tuple = []
for i in new_data0.columns:
    for j in data_MSI_list:
        if i == j[1]:
            dif_MSI_tuple.append(j)
            break
len(dif_MSI_tuple) #296
# dif_MSI_tuple

296

In [38]:
sample_rowname = new_data1._stat_axis.values.tolist() # row name
len(sample_rowname)

296

In [39]:
# put in order
sample_msi_char = []
for i in sample_rowname:
    for j in dif_MSI_tuple:
        if i == j[1]:
            sample_msi_char.append(j[0])
            break
len(sample_msi_char) #296
sample_msi_char

['MSS',
 'MSS',
 'MSS',
 'MSI-H',
 'MSS',
 'MSS',
 'MSS',
 'MSI-H',
 'MSS',
 'MSS',
 'MSS',
 'MSI-H',
 'MSS',
 'MSS',
 'MSS',
 'MSI-H',
 'MSS',
 'MSI-H',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSI-H',
 'MSS',
 'MSI-H',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSI-H',
 'MSI-H',
 'MSI-H',
 'MSI-H',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSI-H',
 'MSS',
 'MSS',
 'MSS',
 'MSI-H',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSI-H',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSI-H',
 'MSS',
 'MSS',
 'MSS',
 'MSI-H',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSI-H',
 'MSS',
 'MSS',
 'MSS',
 'MSI-H',
 'MSS',
 'MSS',
 'MSI-H',
 'MSS',
 'MSS',
 'MSS',
 'MSI-H',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSI-H',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS',
 'MSS'

In [40]:
# Insert the MSI of each sample into it
new_data1.insert(0,"Group",sample_msi_char)
new_data1

,Group,cg09969830,cg00179196,cg03948744,cg02729269,cg10009236,cg10143220,cg05791870,cg01527023,cg00928894,...,cg14029254,cg02352633,cg05051262,cg10666546,cg25640065,cg25918849,cg11021362,cg25914522,cg21106100,cg14273923
TCGA-G4-6315-01A,MSS,0.011653,0.978516,0.021693,0.147438,0.014975,0.939172,0.987482,0.015901,0.067963,...,0.908654,0.901336,0.892368,0.954866,0.955910,0.212444,0.929874,0.939430,0.955336,0.910819
TCGA-AZ-6608-01A,MSS,0.013177,0.972183,0.028394,0.180064,0.019211,0.962210,0.980854,0.016335,0.079980,...,0.788584,0.903834,0.778873,0.962001,0.927002,0.199467,0.904817,0.817178,0.831016,0.868006
TCGA-AY-A8YK-01A,MSS,0.012054,0.973728,0.044442,0.107630,0.016470,0.962572,0.984566,0.016827,0.091078,...,0.904180,0.878067,0.887747,0.943232,0.902062,0.132793,0.867440,0.948413,0.916020,0.901096
TCGA-NH-A5IV-01A,MSI-H,0.014371,0.961363,0.041549,0.160063,0.017490,0.901776,0.980618,0.026201,0.112854,...,0.816789,0.896285,0.803585,0.963357,0.703756,0.177622,0.892458,0.798545,0.912893,0.871487
TCGA-D5-5537-01A,MSS,0.017740,0.961283,0.059346,0.256496,0.029298,0.964023,0.976251,0.022938,0.061267,...,0.679239,0.893329,0.556286,0.961690,0.921391,0.107743,0.864028,0.917506,0.942137,0.915780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-CM-6169-01A,MSS,0.017859,0.952880,0.037437,0.154822,0.024580,0.930985,0.979309,0.021301,0.063823,...,0.836950,0.885340,0.810845,0.939489,0.841554,0.266704,0.877621,0.919844,0.910882,0.814834
TCGA-A6-2686-01A,MSI-H,0.015870,0.961657,0.046022,0.194068,0.021561,0.953753,0.982934,0.020668,0.074079,...,0.845077,0.882007,0.743532,0.923484,0.921326,0.157803,0.894761,0.744305,0.913533,0.862811
TCGA-DM-A285-01A,MSS,0.014479,0.970165,0.037043,0.185207,0.019798,0.907580,0.984501,0.019738,0.116022,...,0.781015,0.881488,0.779299,0.958641,0.895560,0.204045,0.884583,0.665550,0.899826,0.859715
TCGA-DM-A28C-01A,MSS,0.010706,0.977964,0.026631,0.202950,0.016957,0.952098,0.987829,0.021578,0.084261,...,0.960349,0.908695,0.884650,0.965297,0.951437,0.091617,0.914033,0.941300,0.958745,0.885484


In [41]:
new_data1.to_csv("../data/All_Data.csv", sep=',', encoding='utf-8')